In [ ]:
import pandas as pd

# TODO: Cargar con la librería pandas el dataset bank.csv y mostrarlo
# TIP: Al subir el dataset a Google Colab, por defecto se encuentra en la ruta /content/bank.csv
bank_data = pd.read_csv("/content/bank.csv", delimiter=";")
bank_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [ ]:
# TODO: Eliminar la columna "y" del dataset para almacenar el resultado en X (predictores)
# TODO: Guardar la columna "y" en la variable y, y convertir los "yes" en 1s y los "no" en 0s.

X = bank_data.drop("y", axis=1)
y = bank_data["y"].map({"yes": 1, "no": 0})

# TODO: Mostrar la cardinalidad de X e y.

print(f"Cardinalidad de X: {X.shape}")
print(f"Cardinalidad de y: {y.shape}")

Cardinalidad de X: (4521, 16)
Cardinalidad de y: (4521,)


In [ ]:
# TODO: Dividir los datos en train y test.
# TODO: Tamaño de train: 80%
# TODO: Tamaño de test: 20%
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
# TODO: Seleccionar las variables numéricas y las categóricas de los predictores (X_train)
# TIP: Utilizar la función "select_dtypes" de Pandas

num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

In [ ]:
# TODO: Instanciar un transformador StandardScaler para las variables numéricas y un OneHotEncoder para las variables categóricas
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

In [ ]:
# TODO: Importar la clase ColumnTransformer de scikit-learn
from sklearn.compose import ColumnTransformer

# TODO: Investigar como usar la clase ColumnTransformer para aplicar el num_transformer
# a las variables numéricas (num_features) y el cat_transformer a las variables categóricas (cat_features)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

In [ ]:
# TODO: Crear un pipeline con las fases preprocessor y SVC
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

Hiperparámetros:

**Parámetro `C` (Parámetro de regularización)**:
Es el parámetro de regularización del modelo SVM. Esto nos permite evitar el overfitting al penalizar a los modelos más complejos. Un menor valor de C implica un valor de regularización más alto. En resumen, valores muy altos de C regularizan poco el aprendizaje del modelo y menos evitan el overfitting.

**Parámetro `kernel`**: El kernel en una SVM transforma los datos de entrada a un espacio de mayor dimensión donde se espera que los datos sean más fáciles de separar linealmente. Los kernels probados son:
- Linear: No realiza ninguna transformación no lineal sobre los datos, y la separación entre las clases se intenta con un hiperplano lineal.
- RBF (Radial Basis Function): Kernel no lineal que puede resolver el caso donde las clases no son linealmente separables. El Kernel RBF mapea los puntos de entrada a un espacio de características infinito.

In [ ]:
# TODO: Crear una rejilla de hiperparámetros con los siguientes valores posible:
# C: 0.1, 1, 10
# kernel: linear, rbf

param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf']
}

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

CPU times: user 13.3 s, sys: 143 ms, total: 13.5 s
Wall time: 13.5 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object'))])),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__kernel': ['linear', 'rbf']},
             scoring='accuracy')

In [ ]:
print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")

Mejores hiperparámetros encontrados: {'classifier__C': 1, 'classifier__kernel': 'rbf'}


In [ ]:
# TODO: Obtener las predicciones y el classificacion_report
# TODO: Modifica la rejilla de hiperparámetros a ver si eres capaz de encontrar un mejor modelo!
from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       807
           1       0.52      0.22      0.31        98

    accuracy                           0.89       905
   macro avg       0.72      0.60      0.63       905
weighted avg       0.87      0.89      0.87       905

